
# Jacobian for homogenous gas reactor
***Energy equation***
$$
\frac{dT}{dt}= -\frac{1}{\rho c_p}\sum_{k=1}^{N_{spec}}\dot{\omega_{k}} W_k h_k = S_T
$$
***Species equation***
$$
\frac{dY_k}{dt}=\frac{1}{\rho}\dot{\omega_{k}}W_k=S_{Y_k},\,\,\,k=1\ldots N_{spec}
$$


## Environment Setup

In [1]:
TChem_install_directory ='/Users/odiazib/TEST_BLD/install/tchem'

In [2]:
import sys
sys.path.append(TChem_install_directory+'/lib')
import pytchem
import numpy as np
import matplotlib.pyplot as plt
import time

## Pre-simulation
* Set variables; temperature and stoichiometric ratio (fuel/air).
* Convert from stoichiometric ratio to mass fraction of CH4, O2, N2 and AR.
* Create samples spaning over the variable ranges.

In [3]:
# Pressure, Temperature, and mass fractions
one_atm = 101325 # [Pa]
T = 1200 # K 
P = one_atm # Pa
Yp_fuel = 0.1
Yr_o2 = 0.9

In [4]:
N=1
Nvar = 4
sample = np.zeros([N,Nvar])

In [5]:
# sample 1
sample[0,0] = T
sample[0,1] = P
sample[0,2] = Yp_fuel
sample[0,3] = Yr_o2

## TChem Simulation

### Initialize TChem Driver Object

* Initialization of Kokkos.
* Create a TChem driver object. 

In [6]:
pytchem.initialize()

In [7]:
tchem = pytchem.TChemDriver()

* Get help from TChem driver object.

In [8]:
help(tchem)

Help on TChemDriver in module pytchem object:

class TChemDriver(pybind11_builtins.pybind11_object)
 |  A class to manage data movement between numpy to kokkos views in TChem::Driver object
 |  
 |  Method resolution order:
 |      TChemDriver
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: pytchem.TChemDriver) -> None
 |  
 |  computeEnthapyMass(...)
 |      computeEnthapyMass(self: pytchem.TChemDriver) -> None
 |      
 |      Compute enthalpy mass and mixture enthalpy
 |  
 |  computeJacobianHomogeneousGasReactor(...)
 |      computeJacobianHomogeneousGasReactor(self: pytchem.TChemDriver) -> None
 |      
 |      Compute Jacobian matrix for homogeneous gas reactor
 |  
 |  computeNetProductionRatePerMass(...)
 |      computeNetProductionRatePerMass(self: pytchem.TChemDriver) -> None
 |      
 |      Compute net production rate
 |  
 |  computeRHS_HomogeneousGasReactor(...)
 |      computeR

### Create Kinetic Model 

* Inputs are the reactions mechanism files; in this case, we use the GRI3.0 gas reaction mechanism

In [9]:
inputs_directory = TChem_install_directory + '/example/data/H2/'
tchem.createKineticModel(inputs_directory+'chem.inp',inputs_directory+'therm.dat')

### Set State Vector

* Get index for variables. 
* Pass a numpy array to the TChem object to set the state vector. 

In [10]:
Variables = ['T','P','H2','O2']
indx=[]
for var in Variables:
    indx += [tchem.getStateVariableIndex(var)]

state = np.zeros([N, tchem.getLengthOfStateVector()])
for sp in range(N):
    state[sp,indx] = sample[sp,:]

In [11]:
tchem.setNumberOfSamples(N)
tchem.createStateVector()
tchem.setStateVector(state)

In [12]:
tchem.computeJacobianHomogeneousGasReactor()

### Jacobian of Homogeneous Gas Reactor's RHS

In [13]:
# get jacobian for sample 1
jacobian1 = tchem.getJacobianHomogeneousGasReactor(0)

In [14]:
print(jacobian1)

[[-2.58345151e+00  3.25805619e+02 -3.62006243e+01 -7.56177571e+08
   2.17834158e+10  2.22143362e+02 -2.73200920e+09 -2.54627165e+06
  -3.09404524e+07]
 [-5.79033697e-05 -1.07507891e-02 -2.10674121e-03 -4.94015786e+05
  -1.75716792e+06  2.12172226e-03  3.79213460e-02 -1.60025933e+02
   1.12373574e-03]
 [-9.19093759e-04 -1.70646045e-01 -3.34400793e-02  3.79213433e-02
  -7.62422727e-03  3.34842136e-02 -2.87152595e+07  6.63026992e+01
  -7.36375328e+00]
 [ 8.12846431e-13  3.52077758e-11  1.81936607e-11 -3.92072989e+06
   2.16490779e-02  7.28986233e-11  1.13232802e+07 -3.79251009e-12
  -3.68012539e-12]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.16773778e+06
  -1.48242532e+07  1.02942976e-04  1.20366523e+07  0.00000000e+00
   1.23521650e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.57028372e+07 -1.09044062e-04  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.89516923e-05  5.37539709e-03  1.05337056e-03  2.47007893e+05
   8.78583960e+05 -1.0608602

In [15]:
tchem.computeRHS_HomogeneousGasReactor()

In [16]:
tchem.getRHS_HomogeneousGasReactor(0)

array([-1.33285683e+02, -2.97114600e-03, -4.71606759e-02,  1.98950722e-11,
        0.00000000e+00,  0.00000000e+00,  1.48557322e-03,  4.86462486e-02,
        0.00000000e+00])

### Compute Forward and reverse rate constants per reaction 

In [17]:
tchem.computeReactionRateConstants()

The forward rate constant has typically an Arrhenius expression,
$$
{k_f}_i=A_iT^{\beta_i}\exp\left(-\frac{E_i}{RT}\right),
$$
where $A_i$, $\beta_i$, and $E_i$ are the pre-exponential factor, temperature exponent, and activation energy, respectively, for reaction $i$. 

In [18]:
kfor = tchem.getForwardReactionRateConstants()

In [19]:
print('Forward rate constants in units of mol, cm3, s :\n',kfor)

Forward rate constants in units of mol, cm3, s :
 [[1.94132955e+11 6.04973630e+11 2.28740121e+12 3.16043955e+12
  2.16916720e-04 1.77968220e+14 3.92833333e+15 1.55555556e+16
  2.61726559e+15 2.71391201e+13 1.17142081e+14 2.14055791e+13
  1.20833333e+13 1.68600450e+12 6.21438578e+08 2.21914434e+12
  1.71864016e+12 2.60222849e+12 3.84295223e+12]]


For reactions with reverse Arrhenius parameters specified, the reverse rate constant ${k_r}_i$ is computed similar to ${k_f}_i$. If the reverse Arrhenius parameters are not specified, ${k_r}_i$ is computed as
$$
{k_r}_i={k_f}_i/{K_c}_i,
$$
where ${K_c}_i$ is the equilibrium constant (in concentration units) for reaction $i$
$$
{K_c}_i=\left(\frac{P_{atm}}{RT}\right)^{\sum_{k=1}^{N_{spec}}\nu_{ki}}{K_p}_i,\,\,\,
{K_p}_i=\exp\left(\sum_{k=1}^{N_{spec}}\nu_{ki}\left(\frac{S_k}{R}-\frac{H_k}{RT}\right)\right).
$$

In [20]:
krev = tchem.getReverseReactionRateConstants()

In [21]:
print('Reverse rate constants in units of mol, cm3, s :\n',krev)

Reverse rate constants in units of mol, cm3, s :
 [[1.19453660e+13 5.94091851e+11 1.88549537e+10 2.65285468e+10
  5.53980281e+14 1.11213660e-06 1.51051209e-03 5.02073585e-05
  3.43981629e+06 8.08550280e+03 2.10882991e+06 6.26261050e+03
  2.96743455e+01 2.13533735e+06 6.72096456e+15 7.74641558e-02
  4.04285772e+08 6.01126786e+08 7.45163564e+06]]


Finalize Kokkos. This deletes the TChem object and the data stored as Kokkos views

In [22]:
pytchem.finalize()